In [47]:
from PIL import Image
from PIL import ImageFilter
import numpy as np
from scipy.sparse import csr_matrix
from scipy.ndimage.morphology import binary_dilation
from scipy.ndimage import generate_binary_structure
mask_file = 'flow_warped_att00001.png'

In [48]:
def get_rand_att_from_edge(edge_obj, num_edge_points_max):
    edge_arr = np.array(edge_obj, np.uint8)
    true_indices = np.nonzero(edge_arr)
    num_true = true_indices[0].shape[0]
    num_rand_shape = np.random.randint(0,num_edge_points_max+1)
    rand_indices = np.random.choice(num_true, num_rand_shape)
    if rand_indices.size != 0:
        data = np.ones(rand_indices.size, np.uint8)
        row_ind = []
        col_ind = []
        for idx in rand_indices:
            row_ind.append(true_indices[0][idx])
            col_ind.append(true_indices[1][idx])
        sparse_mat = csr_matrix((data, (row_ind, col_ind)), shape=edge_arr.shape)
        sparse_arr = sparse_mat.toarray().astype(np.uint8)
        struct1 = generate_binary_structure(2, 2)
        size_att = np.random.randint(9, 36)
        rand_shape_arr = binary_dilation(sparse_arr, structure=struct1, iterations=size_att).astype(sparse_arr.dtype)
    else:
        rand_shape_arr = np.zeros(edge_arr.shape, np.uint8)
        print('No false or rand att.')
    return rand_shape_arr

In [49]:
# Compute random shape
mask_arr = np.array(Image.open(mask_file), np.uint8)
mask_obj = Image.fromarray(mask_arr, mode='L')
edge_obj = mask_obj.filter(ImageFilter.FIND_EDGES)
rand_shape_arr = get_rand_att_from_edge(edge_obj, 5)

# Compute close by random false attention
struct1 = generate_binary_structure(2, 2)
large_dilate = binary_dilation(mask_arr, structure=struct1, iterations=40).astype(mask_arr.dtype)
large_dilate_obj = Image.fromarray(large_dilate)
large_edge_obj = large_dilate_obj.filter(ImageFilter.FIND_EDGES)
false_att_arr = get_rand_att_from_edge(large_edge_obj, 3)

att_rand = mask_arr + false_att_arr + rand_shape_arr
att_rand_obj = Image.fromarray(att_rand, mode='P')
att_rand_obj.putpalette(np.array([0,0,0,150,0,0]))
att_rand_obj.save('att_rand.png')